In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import mlab
from matplotlib import rcParams
from matplotlib.ticker import FuncFormatter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import numpy as np
%matplotlib inline

In [ ]:
df = pd.read_csv("附件.csv",encoding="gbk")#一开始可能要用 gbk 后面重新保存为csv，可以用utf-8了
print("数据总量：%d" % len(df))
df.head()

In [ ]:
# 任务 1.1 对数据作必要的预处理，在报告中列出处理步骤，将处理后的结 果保存为“task1_1.csv”。

In [ ]:
# d = df[df['销售数量'].isnull()] # '销售数量'中的缺失项
d = df[df.isnull().values==True] # 所有行中的空值的行
d

In [ ]:
# df.dropna()
# df = df.drop(df[(df['销售数量']<0.001) | (df['销售数量']>100000)].index) #删除数量不符合常识的数据
# df = df[df['销售金额'] == df['销售数量']* df['商品单价']] #销售数量与销售金额之间关系正确的
# print("数据总量：%d" % len(df))
# df.head()
# # r_d.to_csv("task1_1.csv")#保存

In [ ]:
# 任务 1.2 统计每个大类商品的销售金额，将结果保存为“task1_2.csv”。

In [ ]:
d = df.groupby(['大类名称'],as_index=False)['销售金额'].sum() #as_index不让大类名称作为索引
d

In [ ]:
# 任务 1.3 统计每个中类商品的促销销售金额和非促销销售金额，将结果保 存为“task1_3.csv”。

In [ ]:
ture_df = df[df['是否促销'] == '是'] 
false_df = df[df['是否促销'] == '否'] 
## shoe = t[t['中类名称'] == '拖鞋']
ture_df = ture_df.groupby(['中类名称'],as_index = False)['销售金额'].sum()
false_df = false_df.groupby(['中类名称'],as_index = False)['销售金额'].sum()
print("数据总量：%d" % (len(ture_df) + len(false_df)))



In [ ]:
d = pd.merge(ture_df,false_df,on = "中类名称") #拼接俩个对象的图
d

In [ ]:
# 任务 1.4 统计生鲜类产品和一般产品的每周销售金额，将结果保存为 “task1_4.csv”。

In [ ]:
#销售日期是否连续,并且排除其他类型
date = df.groupby(['商品类型','销售月份','销售日期'],as_index=False)['销售金额'].sum()
date = date[date['商品类型']!= '联营商品' ]
date = date[date['商品类型'] !='12g*8']
date

In [ ]:
#一般商品类型
normal = date[date['商品类型'] == '一般商品']
normal_goods = []
count = 0 #索引
all_weeks = int(len(normal)/7) + 1 
for weeks in range(1,all_weeks + 1): 
    sum_money = 0
    info = {}
    for j in range(0,7):
        if count < len(normal):
            sum_money = sum_money + normal.iloc[count]['销售金额'] #选取date对象的指定数字的行的某个列的值
            str_type = normal.iloc[count]['商品类型']
            count = count + 1
    info['周数'] = weeks 
    info['商品类型'] = str_type
    info['销售金额'] = sum_money
    normal_goods.append(info)
normal_goods = pd.DataFrame(normal_goods)#生成新的DF对象
normal_goods

In [ ]:
#生鲜商品类型
alive = date[date['商品类型'] == '生鲜']
alive_goods = []
count = 0 #索引
all_weeks = int(len(alive)/7) + 1 
for weeks in range(1,all_weeks + 1): 
    sum_money = 0
    info = {}
    for j in range(0,7):
        if count < len(alive):
            sum_money = sum_money + alive.iloc[count]['销售金额'] #选取date对象的指定数字的行的某个列的值
            str_type = alive.iloc[count]['商品类型']
            count = count + 1
    info['周数'] = weeks 
    info['商品类型'] = str_type
    info['销售金额'] = sum_money
    alive_goods.append(info)
alive_goods = pd.DataFrame(alive_goods)
alive_goods

In [ ]:
d = pd.merge(normal_goods,alive_goods,on = "周数")  #拼接
#d.to_csv("task1_4.csv")
d

In [ ]:
# 任务 1.5 统计每位顾客每月的消费额及消费天数，将结果保存为“task1_5.csv”，并在报告中列出用户编号为 0-10 的结果。 

In [ ]:
person_first = df.groupby(['顾客编号','销售月份','销售日期'],as_index=False)['销售金额'].sum()
person_first.rename(columns={'销售金额':'消费额','销售月份':'月份','销售日期':'日期'},inplace=True)#修改DF对象里的列名,True为真正修改
person_month = person_first.groupby(['顾客编号','月份'],as_index=False)['消费额'].sum()
person_month


In [ ]:
dic = { 201501 :'1月', 201502:'2月' , 201503 :'3月' ,201504 :'4月'}
#person_month.replace(dic,inplace = True) #True为确定源数据的修改
pd.set_option('display.max_columns',20)#后者参数设为None显示所有列
pd.set_option('display.max_rows',30)#后者参数设为None显示所有行
person_month = person_month[person_month['顾客编号'] < 11]
person_month

In [ ]:
person_days = person_first.groupby(['顾客编号','月份'],as_index=False)['日期'].count() #对日期次数进行计数
p = person_days[person_days['顾客编号'] < 11]
p

In [ ]:
#任务 2.1 绘制生鲜类商品和一般商品每天销售金额的折线图,并分析比较两类产品的销售状况。

In [ ]:
#因为有一天没有卖生鲜，后面画图时无法共用X轴，所以人为添加一个数据
for i in range(len(normal['销售日期'])):
    if list(alive['销售日期'])[i] != list(normal['销售日期'])[i]:
        print(list(normal['销售日期'])[i])
        break

alive.loc['new'] = ['生鲜','201502','20150229','700.0']#人为添加一个数据

In [ ]:
d= {'商品类型': '生鲜',
        '销售月份': '201502',
        '销售日期':'20150229',
        '销售金额': 0,
       
        }
a= pd.DataFrame.from_dict(d,orient='index').T
              
# alive=alive.append(a) #运行一次后注释
alive[['销售日期']] = alive[['销售日期']].astype(int)
alive= alive.sort_values(by=['销售日期'],ascending=True)
alive

In [ ]:
normal

In [ ]:
plt.rcParams['font.sans-serif'] = ['STHeiti'] #windows用‘SimHei’字体，mac得用‘STHeiti’
#字体报错的具体解决办法 https://blog.csdn.net/alicelmx/article/details/81231803
plt.rcParams['axes.unicode_minus'] = False     # 正常显示负号

rn = normal['销售日期']#横坐标的内容

fig, ax1 = plt.subplots(figsize =(40,15))#生成图的长和宽
x = np.arange(len(rn))
xticks1 = list(rn)

ax1.axes.set_xlabel("日期", fontsize = 35) #设置x
ax1.axes.set_ylabel("销售金额",fontsize = 35)  #设置y

plt.tick_params(axis = 'both',labelsize = 23) #一些组件的位置参数
plt.xticks(1.5 * x,xticks1,rotation = 80)
plt.title("一般商品和生鲜的每日销售额",verticalalignment='bottom',fontsize=30) #标签

# 折线图
z0 = np.array(normal['销售金额'])
z1 = np.array(alive['销售金额'])
p0 = ax1.plot(x-0.15,z0,c='y',marker='o') #黄色
p1 = ax1.plot(x+0.15,z1,c='r',marker='o') #红色


plt.legend((p0[0],p1[0]), ('一般商品', '生鲜'),loc = 2,prop = {'size':35}) #增加图例标示

# 保存成图片
#plt.savefig("每日销售金额.png",format='png', dpi=300)

plt.show()

In [ ]:
# 任务 2.2 按月绘制各大类商品销售金额的占比饼图，并分析其销售状况。

In [ ]:
month = []
for i in range(0, 4):
    month.append(df[df["销售月份"] == (201501 + i)])

sum_money = [] #销售金额
data = []

for i in range(0, 4):
    sum_money.append(
        pd.pivot_table(data = month[i], index=["大类名称"], values=["销售金额"], fill_value=0, aggfunc=[np.sum, len]))
    data.append(pd.DataFrame({"大类名称": sum_money[i].index, "月销售总额": sum_money[i].iloc[:, 0]}))

plt.rcParams['font.sans-serif'] = ['STHeiti']  # 用来正常显示中文标签
labels = []
sizes = []

for i in range(0, 4):
    fig = plt.figure(figsize=(20,10)) #画图尺寸
    labels.append(list(data[i]["大类名称"]))
    sizes.append(list(data[i]["月销售总额"]))
    plt.pie(sizes[i], labels = labels[i],labeldistance = 1.1, autopct = '%1.2f%%', shadow = False, startangle = 90,
                pctdistance = 1.0)    #画饼图（数据，数据对应的标签，百分数保留两位小数点）
    plt.axis('equal')
    plt.title(str(i + 1) + "月销售金额占比")
    plt.legend()
    #fig.savefig("大类商品销售额{0}占比图.png".format(i + 1),format='png',dpi=300)#保存图片
    plt.show()

In [ ]:
#任务 2.3 绘制促销商品和非促销商品销售金额的周环比增长率柱状图。（环比增长率，一般是指和上期相比较的增长率。
#环比增长率=(本期的某个指标的值-上一期这个指标的值)/上一期这个指标的值*100%。）

In [ ]:
data_csv = df.groupby(['销售月份','销售日期','是否促销'],as_index=False)['销售金额'].sum()
day = set(data_csv['销售日期'])
yes = []
weeks = 0
count = 0
for i in range(int(len(day)/7)+1): 
    sum_money = 0
    info = {}
    for j in range(7):
        if count < len(day):
            str_type = data_csv.iloc[count]['是否促销']
            if str_type=='是':
                sum_money = sum_money + data_csv.iloc[count]['销售金额']
            count += 1
    weeks += 1
    info['周数'] = weeks
    info['是否促销'] = '是'
    info['销售金额'] = sum_money
    yes.append(info)

df_yes = pd.DataFrame(yes)
df_yes

In [ ]:
data_csv = df.groupby(['销售月份','销售日期','是否促销'],as_index=False)['销售金额'].sum()
day = set(data_csv['销售日期'])
no = []
weeks = 0
count = 0
for i in range(int(len(day)/7)+1): 
    sum_money = 0
    info = {}
    for j in range(7):
        if count < len(day):
            str_type = data_csv.iloc[count]['是否促销']
            if str_type == '否':
                sum_money = sum_money + data_csv.iloc[count]['销售金额']
            count += 1
    weeks += 1
    info['周数'] = weeks
    info['是否促销'] = '否'
    info['销售金额'] = sum_money
    no.append(info)

df_no = pd.DataFrame(no)
df_no

In [ ]:
#数据导入折线图前的处理
df_yes_data = round(df_yes['销售金额'].pct_change(),3)#pct_change()方法计算当前元素与先前元素之间的百分比变化
df_no_data = round(df_no['销售金额'].pct_change(),3)#保留3位小数
df_yes_data[0] = 0
df_no_data[0] = 0

In [ ]:
plt.rcParams['font.sans-serif'] = ['STHeiti'] # 指定默认字体 
plt.rcParams['axes.unicode_minus']= False     # 正常显示负号

rn = df_yes['周数']#横坐标的内容
vbar1 = df_yes['销售金额']#   2014的条形图的y轴数据,在图上显示时均保留两位
vbar2 = df_no['销售金额']#   2015的条形图的y轴数据
fig, ax1 = plt.subplots(figsize=(20,8))
x = np.arange(len(rn))
y = np.array(list(vbar1))
w = np.array(list(vbar2))
xticks1 = list(rn)

ax1.axes.set_xlabel(u"周数", fontsize=15)
ax1.axes.set_ylabel(u"销售金额",fontsize=15)  # 设置x 坐标label

#柱状图
p1 = plt.bar(x-0.15,y,width = 0.3,align='center',color = 'c',alpha=0.9) #color
p2 = plt.bar(x+0.15,w,width = 0.3,align='center',color = 'g',alpha=0.9)

plt.tick_params(axis = 'both',labelsize=17)
plt.xticks(x,xticks1,size='x-large')
plt.title("促销/非促销商品周环比增长",verticalalignment='bottom',fontsize=17)

# 折线图
ax2 = ax1.twinx()
z = np.array(df_yes_data)

z1 = np.array(df_no_data)
p3 = ax2.plot(x-0.15,z,c='y',marker='o')
p4 = ax2.plot(x+0.15,z1,c='r',marker='o')

#将坐标轴的值变成百分比
def to_percent(temp, position):
    return '%.2f'%(100 * temp) + '%'

plt.gca().yaxis.set_major_formatter(FuncFormatter(to_percent))

#增加图例
plt.legend((p1[0], p2[0],p3[0],p4[0]),('促销商品', '非促销商品','促销商品周环比增长','非促销商品周环比增长')
           ,loc = 2,prop = {'size':15})

#plt.savefig("环比.png",format = 'png',dpi =300)#保存图片
plt.show()


In [ ]:
# 任务 3.1 根据消费情况，分别为累计消费前 10 的顾客画像。

In [ ]:
consume_top = pd.pivot_table(data = df,index=["顾客编号"],values=["销售金额"],fill_value=0,aggfunc=[np.sum,len])
#列出每名顾客的消费总金额与消费次数
content = pd.DataFrame({"消费总额":consume_top.iloc[:,0]}) #将销售金额切片列出，改为消费总额
consume_sort = content.sort_values(by="消费总额",ascending = False) #降序排列
consume_top10 = consume_sort.head(10)
head_top10 = list(consume_top10.index) # #获取前10编号
person_top10=[]
#获取这些顾客的消费信息
for i in range(0,len(head_top10)):
    person_top10.append(df[df["顾客编号"] == head_top10[i]])

object_top10 = []
object_list = []

for i in range(0,len(head_top10)):
    object_top10 . append(pd.pivot_table(data = person_top10[i],index=["小类名称"],values=["销售金额"],fill_value=0,aggfunc=[np.sum,len]))
    object_list .append(pd.DataFrame({"消费总额":object_top10 [ i].iloc[:,0],"消费数量":object_top10 [ i].iloc[:,1]}))
    print(object_list [i])

#画出按照顾客购买物比列的词图


plt.rcParams['font.sans-serif'] = ['STHeiti'] #用来正常显示中文标签
font  =  r'/System/Library/AssetsV2/com_apple_MobileAsset_Font6/bf625d290b705582d5fd619878281f3325b075d0.asset/AssetData/STHEITI.ttf'
wc  =  WordCloud(font_path  = font, #如果是中文必须要添加这个，否则会显示成框框
                background_color  =  'white',
                max_words  =  200, # 最多显示词数
                max_font_size  =  500, # 字体最大值
                width = 1000,
                height = 800,
                )
for i in range(0,len(head_top10)):#词云图
        wc.generate_from_frequencies(object_list[i]["消费总额"])
        plt.title("顾客" + str(head_top10[i]) + "消费种类占比")
        plt.imshow(wc)  #用plt显示图片
        plt.axis('off') #不显示坐标轴
        plt.savefig("{0}顾客消费种类占比.png".format(head_top10[i]),format='png',dpi = 300)#保存图片,dpi为图片大小参数
        plt.show()

In [ ]:
# 任务 3.2 分析各大类商品的销售情况，总结其销售规律。

In [ ]:
plt.rcParams['font.sans-serif'] = ['STHeiti'] #windows用‘SimHei’字体，mac得用‘STHeiti’
#字体报错的具体解决办法 https://blog.csdn.net/alicelmx/article/details/81231803
plt.rcParams['axes.unicode_minus'] = False     # 正常显示负号

rn = normal['销售日期']#横坐标的内容

fig, ax1 = plt.subplots(figsize =(40,15))#生成图的长和宽
x = np.arange(len(rn))
xticks1 = list(rn)

ax1.axes.set_xlabel("日期", fontsize = 35) #设置x
ax1.axes.set_ylabel("销售金额",fontsize = 35)  #设置y

plt.tick_params(axis = 'both',labelsize = 23) #一些组件的位置参数
plt.xticks(1.5 * x,xticks1,rotation = 80)
plt.title("一般商品和生鲜的每日销售额",verticalalignment='bottom',fontsize=30) #标签

# 折线图
z0 = np.array(normal['销售金额'])
z1 = np.array(alive['销售金额'])
p0 = ax1.plot(x-0.15,z0,c='y',marker='o') #黄色
p1 = ax1.plot(x+0.15,z1,c='r',marker='o') #红色


plt.legend((p0[0],p1[0]), ('一般商品', '生鲜'),loc = 2,prop = {'size':35}) #增加图例标示

# 保存成图片
#plt.savefig("每日销售金额.png",format='png', dpi=300)

plt.show()

In [ ]:
# df  =  pd.DataFrame(df)
print(df.columns)
data_csv_bigclass  =  df['大类名称']
print(data_csv_bigclass)
# 通过集合去重获取大类的名称
big_class_set  =  list(set(data_csv_bigclass))

# 循环分类 总表分割获取各个大类独自对应的内容
save_class  =  []
for i in range(0, len(big_class_set)):
    save_class.append(df[df['大类名称']  ==  big_class_set[i]])
data_all_list  =  []
merge_data  =  []

# 计算各个大类的每日总销售额
for i in range(0, len(big_class_set)):  # values = ['销售金额']可以获得当日销售总额
    data_all_list.append(
        pd.pivot_table(data = save_class[i], index = ['销售日期'], values = ['销售数量'], fill_value = 0, aggfunc = [np.sum, len]))
# print(data_all_list)

# -----------------------画图-------------------------------
plt.rcParams['font.sans-serif']  =  ['STHeiti']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus']  =  False  # 用来正常显示负号
# 循环画图 根据类的个数
for i in range(0, len(data_all_list)):
    # 日期用sort_value()排序
    count_list = []
    for j in range(0,len(data_all_list[i].index.sort_values())):
        count = data_all_list[i].index.sort_values()[j] # max == 115
        count = str(count)
        count = count[-3:]
        count_list.append(count)#将日期切片为更小的数字，方便在坐标轴上呈现。
    plt.subplots(figsize =(50,20))#生成图的长和宽
    plt.xlabel('日期',fontsize = 35)
    plt.xticks(rotation = 80) #rotation为x轴刻度字体的旋转角度
    plt.ylabel('{0}的每日销售数量'.format(big_class_set[i]),fontsize = 35)
    plt.title('{0}的销售总数量折线图'.format(big_class_set[i]),fontsize = 35)
    plt.tick_params(axis = 'both',labelsize = 23) #一些组件的位置参数
    plt.plot(count_list, data_all_list[i].iloc[:, 0], color = 'r', label = big_class_set[i],
             linewidth = 1,marker = 'o', markerfacecolor = 'red', markersize = 1)
#     plt.savefig("{0}.png".format(big_class_set[i]),format='png',dpi = 300)#保存图片 
    #dpi为图像分辨率，
#     plt.show()
print('--------------------------------------------------------------')
# print(count)


In [ ]:
# 任务 3.3 分析促销对商品销售的影响，为超市制定销售策略提供建议。

In [ ]:
sale_big_type  =  df[df["是否促销"]  ==  "是"]
sale  =  pd.pivot_table(data = sale_big_type, index = ["大类名称"], values = ["销售金额"], fill_value = 0, aggfunc = [np.sum, len])
sale_name  =  list(sale.index)
print(sale_name)

sale_yes  =  []
sale_no  =  []
yes_money  =  []
no_money  =  []
for i in range(0, len(sale_name)):
    sale_yes.append(df[(df["大类名称"]  ==  sale_name[i]) & (df["是否促销"]  ==  "是")])
    riqi1  =  len(sale_yes[i]["销售日期"].to_list())
    yes_money.append(sale_yes[i]["销售金额"].sum() / riqi1)

    sale_no.append(df[(df["大类名称"]  ==  sale_name[i]) & (df["是否促销"]  ==  "否")])
    riqi2  =  len(sale_no[i]["销售日期"].to_list())
    no_money.append(sale_no[i]["销售金额"].sum() / riqi2)

print(yes_money)
print('\n')
print(no_money)

# -----------------------画图-------------------------------
plt.rcParams['font.sans-serif']  =  ['STHeiti']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus']  =  False  # 用来正常显示负号

bar_width  =  0.4
# 将X轴数据改为使用range(len(x_data), 就是0、1、2...
plt.bar(sale.index, height = no_money, label = '促销日均销售金额', color = 'indianred', alpha = 0.8, width = bar_width)
# 将X轴数据改为使用np.arange(len(x_data))+bar_width,
# 就是bar_width、1+bar_width、2+bar_width...这样就和第一个柱状图并列了
plt.bar(x = np.arange(0, 10) + bar_width, height = yes_money, label = '非促销日均销售金额', color = 'steelblue', alpha = 0.8,
        width = bar_width)

# 设置标题
plt.title("促销日均销售金额和非促销日均销售金额对比")
# 为两条坐标轴设置名称
plt.xlabel("大类名称")
plt.ylabel("日均销售金额")
plt.xlim((-1, 11))

# 显示图例
plt.legend()
plt.show()
